<img src="http://hauteliving.com/wp-content/uploads/2012/03/lc.jpg">

### This jupyter notebook aims to build a baseline model for restaurant visitor count forecast problem using Linear Regression(Ordinary Least Squares).

In [116]:
#Load train, test data to dataFrames
%matplotlib inline
import pandas as pd
import numpy as np
import re
import warnings
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
import seaborn as sns
plt.style.use('ggplot')
warnings.simplefilter("ignore")
df_train = pd.read_csv(".//Data//Train_data.csv")
df_test = pd.read_csv(".//Data//Test_data.csv")
df_date_info = pd.read_csv('.//Data//date_info.csv')

df_date_info.calendar_date = pd.to_datetime(df_date_info.calendar_date)
#change the index of df_date_info to calendar_date
df_date_info = df_date_info.set_index('calendar_date')
df_train['holiday_flg'] = df_train['visit_date'].apply(lambda x : df_date_info.loc[x]['holiday_flg'])
df_test['holiday_flg'] = df_test['visit_date'].apply(lambda x : df_date_info.loc[x]['holiday_flg'])

In [117]:
df_train.head(5)

,air_store_id,visit_date,visitors,dow,year,month,mean_visitors,max_visitors,min_visitors,median_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y,holiday_flg
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,23.843750,57.0,7.0,25.0,Dining bar,T?ky?-to Minato-ku Shibak?en,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,0
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,20.292308,54.0,2.0,21.0,Dining bar,T?ky?-to Minato-ku Shibak?en,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,0
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,34.738462,61.0,4.0,35.0,Dining bar,T?ky?-to Minato-ku Shibak?en,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,0
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,27.651515,53.0,6.0,27.0,Dining bar,T?ky?-to Minato-ku Shibak?en,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,0
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,13.754386,34.0,2.0,12.0,Dining bar,T?ky?-to Minato-ku Shibak?en,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,0


In [118]:
df_test.head(5)

,id,visitors,visit_date,air_store_id,dow,year,month,mean_visitors,max_visitors,min_visitors,median_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y,holiday_flg
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,2.000000,2.0,2.0,2.0,Italian/French,T?ky?-to Chiyoda-ku Kudanminami,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0,0
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,22.457143,47.0,1.0,19.0,Italian/French,T?ky?-to Chiyoda-ku Kudanminami,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0,0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,24.350000,43.0,1.0,24.5,Italian/French,T?ky?-to Chiyoda-ku Kudanminami,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0,0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,28.125000,52.0,15.0,28.0,Italian/French,T?ky?-to Chiyoda-ku Kudanminami,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0,0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,29.868421,47.0,15.0,30.0,Italian/French,T?ky?-to Chiyoda-ku Kudanminami,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0,0


##### Change the columns to their respective types
#####   air_area_name =====> categorical type
#####   air_genre_name ===> categorical type 

In [119]:
#Change latitude,longitude type to float type
df_train['air_genre_name'] = df_train['air_genre_name'].apply(lambda x : 'Other' if x == '-1' else x)
df_test['air_genre_name'] = df_test['air_genre_name'].apply(lambda x : 'Other' if x == '-1' else x)


df_train['latitude'] = df_train['latitude'].convert_objects(convert_numeric=True)
df_train['longitude'] = df_train['longitude'].convert_objects(convert_numeric=True)

df_test['latitude'] = df_test['latitude'].convert_objects(convert_numeric=True)
df_test['longitude'] = df_test['longitude'].convert_objects(convert_numeric=True)

#change the air_genre_name,air_store_id to categorical type
df_train['air_area_name'] = df_train['air_area_name'].astype('category')
df_train['air_genre_name'] = df_train['air_genre_name'].astype('category')

df_test['air_area_name'] = df_test['air_area_name'].astype('category')
df_test['air_genre_name'] = df_test['air_genre_name'].astype('category')

In [120]:
df_train['visit_date'] = pd.to_datetime(df_train['visit_date'])
df_test['visit_date'] = pd.to_datetime(df_test['visit_date'])

df_train['day'] = df_train['visit_date'].dt.day
df_test['day'] = df_test['visit_date'].dt.day

df_train['fri'] = (df_train['dow'] == 4)*1
df_train['sat'] = (df_train['dow'] == 5)*1
df_train['sun'] = (df_train['dow'] == 6)*1

df_test['fri'] = (df_test['dow'] == 4)*1
df_test['sat'] = (df_test['dow'] == 5)*1
df_test['sun'] = (df_test['dow'] == 6)*1


In [121]:
len(df_train['air_genre_name'].unique())

14

In [128]:
from sklearn.preprocessing import LabelEncoder
le_train = LabelEncoder()
le_train.fit(df_train['air_store_id'].values)

LabelEncoder()

In [129]:
df_train['air_store_id'] = le_train.transform(df_train['air_store_id'].values)
df_test['air_store_id'] = le_train.transform(df_test['air_store_id'].values)

In [130]:
df_train.head()

,air_store_id,visit_date,visitors,dow,year,month,mean_visitors,max_visitors,min_visitors,median_visitors,...,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y,holiday_flg,day,fri,sat,sun
0,603,2016-01-13,25,2,2016,1,23.843750,57.0,7.0,25.0,...,139.751599,-1.0,-1.0,-1.0,-1.0,0,13,0,0,0
1,603,2016-01-14,32,3,2016,1,20.292308,54.0,2.0,21.0,...,139.751599,-1.0,-1.0,-1.0,-1.0,0,14,0,0,0
2,603,2016-01-15,29,4,2016,1,34.738462,61.0,4.0,35.0,...,139.751599,-1.0,-1.0,-1.0,-1.0,0,15,1,0,0
3,603,2016-01-16,22,5,2016,1,27.651515,53.0,6.0,27.0,...,139.751599,-1.0,-1.0,-1.0,-1.0,0,16,0,1,0
4,603,2016-01-18,6,0,2016,1,13.754386,34.0,2.0,12.0,...,139.751599,-1.0,-1.0,-1.0,-1.0,0,18,0,0,0


In [132]:
df_test.head()

,id,visitors,visit_date,air_store_id,dow,year,month,mean_visitors,max_visitors,min_visitors,...,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y,holiday_flg,day,fri,sat,sun
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,0,6,2017,4,2.000000,2.0,2.0,...,139.753595,-1.0,-1.0,-1.0,-1.0,0,23,0,0,1
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,0,0,2017,4,22.457143,47.0,1.0,...,139.753595,-1.0,-1.0,-1.0,-1.0,0,24,0,0,0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,0,1,2017,4,24.350000,43.0,1.0,...,139.753595,-1.0,-1.0,-1.0,-1.0,0,25,0,0,0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,0,2,2017,4,28.125000,52.0,15.0,...,139.753595,-1.0,-1.0,-1.0,-1.0,0,26,0,0,0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,0,3,2017,4,29.868421,47.0,15.0,...,139.753595,-1.0,-1.0,-1.0,-1.0,0,27,0,0,0


In [133]:
df_train_with_dummies = pd.get_dummies(df_train.drop('visit_date',axis=1))
df_train_with_dummies.head()

,air_store_id,visitors,dow,year,month,mean_visitors,max_visitors,min_visitors,median_visitors,latitude,...,air_area_name_T?ky?-to Suginami-ku Asagayaminami,air_area_name_T?ky?-to Tachikawa-shi Izumich?,air_area_name_T?ky?-to Tait?-ku Asakusa,air_area_name_T?ky?-to Tait?-ku Higashiueno,air_area_name_T?ky?-to Tait?-ku Kuramae,air_area_name_T?ky?-to Toshima-ku Higashiikebukuro,air_area_name_T?ky?-to Toshima-ku Mejiro,air_area_name_T?ky?-to Toshima-ku Minamiikebukuro,air_area_name_T?ky?-to Toshima-ku Nishiikebukuro,air_area_name_T?ky?-to Toshima-ku Sugamo
0,603,25,2,2016,1,23.843750,57.0,7.0,25.0,35.658068,...,0,0,0,0,0,0,0,0,0,0
1,603,32,3,2016,1,20.292308,54.0,2.0,21.0,35.658068,...,0,0,0,0,0,0,0,0,0,0
2,603,29,4,2016,1,34.738462,61.0,4.0,35.0,35.658068,...,0,0,0,0,0,0,0,0,0,0
3,603,22,5,2016,1,27.651515,53.0,6.0,27.0,35.658068,...,0,0,0,0,0,0,0,0,0,0
4,603,6,0,2016,1,13.754386,34.0,2.0,12.0,35.658068,...,0,0,0,0,0,0,0,0,0,0


In [134]:
df_test_with_dummies = pd.get_dummies(df_test.drop(['id','visit_date'],axis=1))
df_test_with_dummies.head()

,visitors,air_store_id,dow,year,month,mean_visitors,max_visitors,min_visitors,median_visitors,latitude,...,air_area_name_T?ky?-to Suginami-ku Asagayaminami,air_area_name_T?ky?-to Tachikawa-shi Izumich?,air_area_name_T?ky?-to Tait?-ku Asakusa,air_area_name_T?ky?-to Tait?-ku Higashiueno,air_area_name_T?ky?-to Tait?-ku Kuramae,air_area_name_T?ky?-to Toshima-ku Higashiikebukuro,air_area_name_T?ky?-to Toshima-ku Mejiro,air_area_name_T?ky?-to Toshima-ku Minamiikebukuro,air_area_name_T?ky?-to Toshima-ku Nishiikebukuro,air_area_name_T?ky?-to Toshima-ku Sugamo
0,0,0,6,2017,4,2.000000,2.0,2.0,2.0,35.694003,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,2017,4,22.457143,47.0,1.0,19.0,35.694003,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,2017,4,24.350000,43.0,1.0,24.5,35.694003,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,2017,4,28.125000,52.0,15.0,28.0,35.694003,...,0,0,0,0,0,0,0,0,0,0
4,0,0,3,2017,4,29.868421,47.0,15.0,30.0,35.694003,...,0,0,0,0,0,0,0,0,0,0


In [135]:
y = np.log1p(df_train_with_dummies['visitors']).values
X = df_train_with_dummies.drop(['visitors','air_area_name_-1'],axis=1).values

In [136]:
# Import regression modules
import pprint
import statsmodels.api as sm
from statsmodels.datasets.longley import load_pandas
from statsmodels.formula.api import ols
ols_model = sm.OLS(np.log1p(df_train_with_dummies['visitors']), df_train_with_dummies.drop(['visitors','air_area_name_-1'],axis=1))
ols_results = ols_model.fit()
pprint.pprint(ols_results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:               visitors   R-squared:                       0.536
Model:                            OLS   Adj. R-squared:                  0.536
Method:                 Least Squares   F-statistic:                     2155.
Date:                Mon, 05 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:20:42   Log-Likelihood:            -2.0713e+05
No. Observations:              252108   AIC:                         4.145e+05
Df Residuals:                  251972   BIC:                         4.159e+05
Df Model:                         135                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [137]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  scale
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
lm = LinearRegression()
lm.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [138]:
y_pred = lm.predict(X_test)
from sklearn.metrics import mean_squared_error
#squared_error = ((y_pred + 1) - (y_test + 1)) ** 2
#rmsle = np.sqrt( np.sum(squared_error) / len(y_pred) )
rmse = np.sqrt(mean_squared_error(y_pred,y_test))
print("RMSE",rmse)
print(max(y_pred))
#print(y_pred)

RMSE 0.550315961322
9.14017796168


In [139]:
print("Training Accuracy : ",lm.score(X_train,y_train))
print("Test Accuracy : ",lm.score(X_test,y_test))

Training Accuracy :  0.534997281519
Test Accuracy :  0.537111657464


In [140]:
df_test_with_dummies.head()

,visitors,air_store_id,dow,year,month,mean_visitors,max_visitors,min_visitors,median_visitors,latitude,...,air_area_name_T?ky?-to Suginami-ku Asagayaminami,air_area_name_T?ky?-to Tachikawa-shi Izumich?,air_area_name_T?ky?-to Tait?-ku Asakusa,air_area_name_T?ky?-to Tait?-ku Higashiueno,air_area_name_T?ky?-to Tait?-ku Kuramae,air_area_name_T?ky?-to Toshima-ku Higashiikebukuro,air_area_name_T?ky?-to Toshima-ku Mejiro,air_area_name_T?ky?-to Toshima-ku Minamiikebukuro,air_area_name_T?ky?-to Toshima-ku Nishiikebukuro,air_area_name_T?ky?-to Toshima-ku Sugamo
0,0,0,6,2017,4,2.000000,2.0,2.0,2.0,35.694003,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,2017,4,22.457143,47.0,1.0,19.0,35.694003,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,2017,4,24.350000,43.0,1.0,24.5,35.694003,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,2017,4,28.125000,52.0,15.0,28.0,35.694003,...,0,0,0,0,0,0,0,0,0,0
4,0,0,3,2017,4,29.868421,47.0,15.0,30.0,35.694003,...,0,0,0,0,0,0,0,0,0,0


In [141]:
df_train_with_dummies.head()

,air_store_id,visitors,dow,year,month,mean_visitors,max_visitors,min_visitors,median_visitors,latitude,...,air_area_name_T?ky?-to Suginami-ku Asagayaminami,air_area_name_T?ky?-to Tachikawa-shi Izumich?,air_area_name_T?ky?-to Tait?-ku Asakusa,air_area_name_T?ky?-to Tait?-ku Higashiueno,air_area_name_T?ky?-to Tait?-ku Kuramae,air_area_name_T?ky?-to Toshima-ku Higashiikebukuro,air_area_name_T?ky?-to Toshima-ku Mejiro,air_area_name_T?ky?-to Toshima-ku Minamiikebukuro,air_area_name_T?ky?-to Toshima-ku Nishiikebukuro,air_area_name_T?ky?-to Toshima-ku Sugamo
0,603,25,2,2016,1,23.843750,57.0,7.0,25.0,35.658068,...,0,0,0,0,0,0,0,0,0,0
1,603,32,3,2016,1,20.292308,54.0,2.0,21.0,35.658068,...,0,0,0,0,0,0,0,0,0,0
2,603,29,4,2016,1,34.738462,61.0,4.0,35.0,35.658068,...,0,0,0,0,0,0,0,0,0,0
3,603,22,5,2016,1,27.651515,53.0,6.0,27.0,35.658068,...,0,0,0,0,0,0,0,0,0,0
4,603,6,0,2016,1,13.754386,34.0,2.0,12.0,35.658068,...,0,0,0,0,0,0,0,0,0,0


In [142]:
X_test.shape

(75633, 136)

In [146]:
df_test_with_dummies.shape

(32019, 137)

In [149]:
#df_test_with_dummies['air_genre_name_-1'] = 0
y_final_pred = lm.predict(df_test_with_dummies.drop(['visitors'],axis=1).values)

In [150]:
y_final_pred

array([ 1.84583384,  2.75780872,  2.89599467, ...,  1.96385806,
        1.99222627,  2.01070391])

In [151]:
df_test['visitors'] = np.expm1(y_final_pred)
df_test['air_store_id'] = le_train.inverse_transform(df_test['air_store_id'].values)

In [153]:
df_test['visitors'] = df_test['visitors'].clip_lower(0.)

In [158]:
df_test[['id','visitors']]

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,5.333379
1,air_00a91d42b08b08d9_2017-04-24,14.765259
2,air_00a91d42b08b08d9_2017-04-25,17.101497
3,air_00a91d42b08b08d9_2017-04-26,19.655015
4,air_00a91d42b08b08d9_2017-04-27,21.835756
5,air_00a91d42b08b08d9_2017-04-28,29.431629
6,air_00a91d42b08b08d9_2017-04-29,11.767954
7,air_00a91d42b08b08d9_2017-04-30,5.433662
8,air_00a91d42b08b08d9_2017-05-01,13.918477
9,air_00a91d42b08b08d9_2017-05-02,16.129232


In [160]:
df_test[['id','visitors']].to_csv('BaseLine_1.csv',index=False)

#### Baseline model score in Kaggle : 0.568